In [1]:
import numpy as np
import os
import pandas as pd
from scipy import signal
from scipy.signal import chirp, find_peaks, peak_widths
import ephyviewer
from ephyviewer import mkQApp, MainViewer, TraceViewer
from ephyviewer import AnalogSignalSourceWithScatter

### Extract relevant variables from recordings

In [2]:
#Variables that might change from mouse to mouse

# format: recordings = [date_time, TTL of midline, TTL of VB]       #put float("nan") if no stim of this type in recording!!
recordings = [["2023-11-14_15-45-09", 1, 2], ["2023-11-15_14-00-12", float("nan"), 1], ["2023-11-16_13-47-23", 1, 2], ["2024-01-23_11-42-06", 2, float("nan")]]
# format: CA1_Eles = [CA1 ipsi, CA1 contra]
CA1_Eles = [7, 14]
stim_regions = ["mid", "vb"]    #make sure to have that in same order as TTLs in recordings info
Letter = "I:"
nb_E = 15
mouse = os.path.basename(os.getcwd())   #script contained in folder named after mouse

#Plenty of empty lists that will be filled with variables
EMGboolean = []
EMGs = []
CA1_i = []
CA1_c = []
CA1_i_cwt = []
CA1_c_cwt = []
CA1_i_means_sd = []
CA1_c_means_sd = []
CA1_i_stim0 = []
CA1_c_stim0 = []
CA1_i_onlystim = []
CA1_c_onlystim = []


for rec_nb, rec_infos in enumerate(recordings):
    print(rec_infos)
    date_time = rec_infos[0]
    if date_time == "2024-01-23_11-42-06":
        recordnode = 101
        acquiboard = 100
    else:
        recordnode = 105
        acquiboard = 104


    ### STIM TIMES ###
    #Load stim info from hard drive with original recording
    sample_numbers = np.load(f"{Letter}/{date_time}/Record Node {recordnode}/experiment1/recording1/events/Acquisition_Board-{acquiboard}.Rhythm Data/TTL/sample_numbers.npy")
    channel_states = np.load(f"{Letter}/{date_time}/Record Node {recordnode}/experiment1/recording1/events/Acquisition_Board-{acquiboard}.Rhythm Data/TTL/states.npy")
    cont = np.load(f"{Letter}/{date_time}/Record Node {recordnode}/experiment1/recording1/continuous/Acquisition_Board-{acquiboard}.Rhythm Data/sample_numbers.npy")

    for pos, stim_region in enumerate(stim_regions):
        TTL = rec_infos[pos+1]  #because loops through mid then vb, and info about TTL is in position 1 and 2 of sub-list in "recordings" (line 4)
        
        if TTL == float("nan"):
            globals()[f"stim_times_{stim_region}_rec{rec_nb}"] = float("nan")

        else:
            # find the indices of the start and end times
            start_indices = np.where(channel_states == TTL)[0]
            end_indices = np.where(channel_states == -TTL)[0]

            # create a new array with the start and end times
            stim_times = np.zeros((len(start_indices), 2))
            for i in range(len(start_indices)):
                stim_times[i, 0] = sample_numbers[start_indices[i]]
                stim_times[i, 1] = sample_numbers[end_indices[i]]

            #Adjust stim timestamps - start time and sampling frequency
            start_time = cont[0]  # in 0.5 milliseconds   #First value of timestamps memmap from 'continuous' folder
            stim_times -= start_time
            stim_times = stim_times.astype('float64')
            stim_times /= 2.0
            stim_times = stim_times.astype('int64')

            #Identify stim_times by name of stimulated region, keep only that
            globals()[f"stim_times_{stim_region}_rec{rec_nb}"] = stim_times; stim_times = None



    ### LFP RECORDINGS ###
    # Load DS and ordered recordings, and sleep scoring frame, from subfolder
    EMGboolean.append(pd.read_pickle(f'{date_time}/EMGframeBoolean_1.pkl'))
    All = np.load(f'{date_time}/RawDataChannelExtractedDS_1.npy', mmap_mode= 'r')

    #Get LFPs by calculating differences between electrode pairs
    EMG = All[:,0]
    y = 1
    for x in range(nb_E):
        exec(f"Ea = All[:,{y}]")
        y += 1
        exec(f"Eb = All[:,{y}]")
        y += 1
        exec(f"E{x} = Ea - Eb")

    #Keep only CA1 channels, erase the rest
    exec(f"CA1_i.append(E{CA1_Eles[0]})")
    exec(f"CA1_c.append(E{CA1_Eles[1]})")

    # Filter EMG for nicer visualisation later on
    f_lowcut = 200.; f_hicut = 400.; N = 4; fs = 1000; nyq = 0.5 * fs
    Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction
    b, a = signal.butter(N, Wn, 'band')
    filt_EMG = signal.filtfilt(b, a, EMG)
    EMGs.append(filt_EMG)
    
    All = None; Ea = None; Eb = None
    for x in range(nb_E):
        exec(f"E{x} = None")
#Gathered all info from recordings, now end loop
    

### ANALYSE CA1 LFPs ###
# Contiuous wavelet transform (parameters and computation)
w = 10.
fs = 1000
freq = np.linspace(120, 200, 80)
widths = w*fs / (2*freq*np.pi)

for rec_nb in range(len(recordings)):
    CA1_i_cwt.append(np.mean(np.absolute(signal.cwt(CA1_i[rec_nb], signal.morlet2, widths, w=w)), axis=0))
    CA1_c_cwt.append(np.mean(np.absolute(signal.cwt(CA1_c[rec_nb], signal.morlet2, widths, w=w)), axis=0))

# From the whole recording, keep only times where there is a stimulation (with some time before and after) that entirely happens during sleep
for hemi in ["i", "c"]:
    for rec_nb in range(len(recordings)):
        CWT_wakeremoved = np.array(globals()[f"CA1_{hemi}_cwt"][rec_nb].copy())
        CWT_wakeremoved = CWT_wakeremoved[EMGboolean[rec_nb]['BooleanLiberal']]  #Keep only SWS
        CA1_mean = np.mean(CWT_wakeremoved)
        CA1_sd = np.std(CWT_wakeremoved)
        globals()[f"CA1_{hemi}_means_sd"].append([CA1_mean, CA1_sd])
        CWT_stim0 = np.array(globals()[f"CA1_{hemi}_cwt"][rec_nb].copy())
        CWT_stim0[~EMGboolean[rec_nb]['BooleanLiberal']] = 0  #Put everything else than SWS to 0

        stimtimes_vector = np.zeros(len(CWT_stim0))
        for stim_region in stim_regions:
            stim_times = globals()[f"stim_times_{stim_region}_rec{rec_nb}"]
            for stim_nb in range(0, len(stim_times), 10):   #every 10th stim because blocks of 10
                stim_start = stim_times[stim_nb, 0]
                stim_end = stim_times[stim_nb+9, 1]
                stim_duration = stim_end-stim_start

                # Marquer la période de stimulation dans le vecteur
                stim_window = slice(stim_start - stim_duration, stim_end + stim_duration + 10000)
                stimtimes_vector[stim_window] = 1

                # Vérifier si au moins une valeur dans la fenêtre de stimulation n'est pas contenue dans EMGboolean-sommeil
                if np.any(~EMGboolean[rec_nb]['BooleanLiberal'][stim_window]):
                    stimtimes_vector[stim_window] = 0

            # # Exclure les périodes marquées par EMGboolean
            stimtimes_vector[~EMGboolean[rec_nb]['BooleanLiberal']] = 0
            tests.append(stimtimes_vector)

        # Créer un vecteur booléen pour les périodes de stimulation
        bool_stimtimes = (stimtimes_vector == 1)
        CWT_stim0[~bool_stimtimes] = 0   #now also put everything that is not before/during/after stim to 0
        globals()[f"CA1_{hemi}_stim0"].append(CWT_stim0)
        times = np.arange(len(CWT_stim0))[bool_stimtimes]
        CWT_onlystim = np.concatenate((CWT_stim0[bool_stimtimes], times)).reshape(2,len(times)) #create variable where everything except valid stimtimes is removed, but add times row to preserve info about original times
        globals()[f"CA1_{hemi}_onlystim"].append(CWT_onlystim)
        globals()[f"bool_stimtimes_rec{rec_nb}"] = bool_stimtimes

CWT_wakeremoved = None #free memory

['2023-11-14_15-45-09', 1, 2]
['2023-11-15_14-00-12', nan, 1]
['2023-11-16_13-47-23', 1, 2]
['2024-01-23_11-42-06', 2, nan]


### Detect SWRs

#### Recording 0

In [3]:
### REC 0 ###

### DETECT SWRs ON STIM ONLY DATA ###
#Do both hemispheres (can view both at same time)
for hemi in ["i", "c"]:
    if hemi == "i":
        threshold = 4   #number by which to multiply SD; 2 diff thresholds for 2 diff channels
    else:
        threshold = 4
    
    CA1_mean = globals()[f"CA1_{hemi}_means_sd"][0][0]
    CA1_sd = globals()[f"CA1_{hemi}_means_sd"][0][1]
    peaks, properties = find_peaks(globals()[f"CA1_{hemi}_onlystim"][0][0], prominence=1, width=20, height = CA1_mean+CA1_sd*threshold)

    properties["prominences"], properties["widths"]
    # SWR boundaries taken at 70% from peak of intensity. This means that the SWRs with small amplitude will be longer than the big ones.
    results_width = peak_widths(globals()[f"CA1_{hemi}_onlystim"][0][0], peaks, rel_height=0.7)
    peaks2 = peaks.reshape(len(peaks),1)
    npresults_width = np.array(results_width).reshape(4,-1)
    SWR_prop = np.append(peaks2, results_width).reshape(5,len(peaks2)).round()

    #Filter signal because is nice for SWR visualisation
    f_lowcut = 120.; f_hicut = 200.; fs = 1000; nyq = 0.5 * fs; N = 6
    Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction
    b, a = signal.butter(N, Wn, 'band')
    globals()[f"filt_CA1_{hemi}"] = signal.filtfilt(b, a, globals()[f"CA1_{hemi}"][0])
    globals()[f"peaks_{hemi}_rec0"] = peaks
    globals()[f"SWR_prop_{hemi}"] = SWR_prop



### LOOK AT THE RESULT AND JUDGE IT; UPDATE THRESHOLD AND REPEAT IF NECESSARY ###
app = mkQApp()

sample_rate = 1000.
t_start = 0.

combined = np.stack([CA1_i[0][bool_stimtimes_rec0], CA1_i_onlystim[0][0], filt_CA1_i[bool_stimtimes_rec0], CA1_c[0][bool_stimtimes_rec0], CA1_c_onlystim[0][0], filt_CA1_c[bool_stimtimes_rec0], EMGs[0][bool_stimtimes_rec0]], axis = 1)

#create 2 family scatters from theses 2 indexes
scatter_indexes = {0: peaks_i_rec0, 1: peaks_c_rec0}
#and assign them to some channels each
scatter_channels = {0: [0, 1], 1: [3, 4]}

source = AnalogSignalSourceWithScatter(combined, sample_rate, t_start, scatter_indexes, scatter_channels)

#Create the main window that can contain several viewers
win = MainViewer(debug=True, show_auto_scale=True)

#create a viewer for signal with TraceViewer
#connected to the signal source
view1 = TraceViewer(source=source)

#Parameters can be set in script
#view1.params['scale_mode'] = 'same_for_all'
view1.params['display_labels'] = True
#And also parameters for each channel
view1.by_channel_params['ch0', 'color'] = '#ffffff'
view1.by_channel_params['ch1', 'color'] = '#0055ff'
view1.by_channel_params['ch2', 'color'] = '#ff5500'
view1.by_channel_params['ch3', 'color'] = '#ffffff'
view1.by_channel_params['ch4', 'color'] = '#0055ff'
view1.by_channel_params['ch5', 'color'] = '#ff5500'
view1.by_channel_params['ch6', 'color'] = 'red'

view1.by_channel_params['ch0', 'gain'] = 0.0005
view1.by_channel_params['ch1', 'gain'] = 0.0015
view1.by_channel_params['ch2', 'gain'] = 0.0015
view1.by_channel_params['ch3', 'gain'] = 0.001
view1.by_channel_params['ch4', 'gain'] = 0.0015
view1.by_channel_params['ch5', 'gain'] = 0.0015
view1.by_channel_params['ch6', 'gain'] = 0.001

view1.by_channel_params['ch0', 'offset'] = 4
view1.by_channel_params['ch1', 'offset'] = 1
view1.by_channel_params['ch2', 'offset'] = 1
view1.by_channel_params['ch3', 'offset'] = -3
view1.by_channel_params['ch4', 'offset'] = -6
view1.by_channel_params['ch5', 'offset'] = -6
view1.by_channel_params['ch6', 'offset'] = 8

#put this viewer in the main window
win.add_view(view1)

#Run
win.show()
app.exec()

print(len(CA1_i_onlystim[0][0]))
print(len(peaks_i_rec0), len(peaks_c_rec0))

debug True
QT_MODE PySide6
refresh duration for  0.0009975433349609375 s
save_all_settings
3107099
1627 1910


#### Recording 1

In [4]:
### REC 1 ###

### DETECT SWRs ON STIM ONLY DATA ###
#Do both hemispheres (can view both at same time) but only one recording at a time
for hemi in ["i", "c"]:
    if hemi == "i":
        threshold = 4   #number by which to multiply SD; 2 diff thresholds for 2 diff channels
    else:
        threshold = 4
    
    CA1_mean = globals()[f"CA1_{hemi}_means_sd"][1][0]
    CA1_sd = globals()[f"CA1_{hemi}_means_sd"][1][1]
    peaks, properties = find_peaks(globals()[f"CA1_{hemi}_onlystim"][1][0], prominence=1, width=20, height = CA1_mean+CA1_sd*threshold)

    properties["prominences"], properties["widths"]
    # SWR boundaries taken at 70% from peak of intensity. This means that the SWRs with small amplitude will be longer than the big ones.
    results_width = peak_widths(globals()[f"CA1_{hemi}_onlystim"][1][0], peaks, rel_height=0.7)
    peaks2 = peaks.reshape(len(peaks),1)
    npresults_width = np.array(results_width).reshape(4,-1)
    SWR_prop = np.append(peaks2, results_width).reshape(5,len(peaks2)).round()

    #Filter signal because is nice for SWR visualisation
    f_lowcut = 120.; f_hicut = 200.; fs = 1000; nyq = 0.5 * fs; N = 6
    Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction
    b, a = signal.butter(N, Wn, 'band')
    globals()[f"filt_CA1_{hemi}"] = signal.filtfilt(b, a, globals()[f"CA1_{hemi}"][1])
    globals()[f"peaks_{hemi}_rec1"] = peaks
    globals()[f"SWR_prop_{hemi}"] = SWR_prop



### LOOK AT THE RESULT AND JUDGE IT; REPEAT IF NECESSARY ###
app = mkQApp()

sample_rate = 1000.
t_start = 0.

combined = np.stack([CA1_i[1][bool_stimtimes_rec1], CA1_i_onlystim[1][0], filt_CA1_i[bool_stimtimes_rec1], CA1_c[1][bool_stimtimes_rec1], CA1_c_onlystim[1][0], filt_CA1_c[bool_stimtimes_rec1], EMGs[1][bool_stimtimes_rec1]], axis = 1)

#create 2 family scatters from theses 2 indexes
scatter_indexes = {0: peaks_i_rec1, 1: peaks_c_rec1}
#and assign them to some channels each
scatter_channels = {0: [0, 1], 1: [3, 4]}

source = AnalogSignalSourceWithScatter(combined, sample_rate, t_start, scatter_indexes, scatter_channels)

#Create the main window that can contain several viewers
win = MainViewer(debug=True, show_auto_scale=True)

#create a viewer for signal with TraceViewer
#connected to the signal source
view1 = TraceViewer(source=source)

#Parameters can be set in script
#view1.params['scale_mode'] = 'same_for_all'
view1.params['display_labels'] = True
#And also parameters for each channel
view1.by_channel_params['ch0', 'color'] = '#ffffff'
view1.by_channel_params['ch1', 'color'] = '#0055ff'
view1.by_channel_params['ch2', 'color'] = '#ff5500'
view1.by_channel_params['ch3', 'color'] = '#ffffff'
view1.by_channel_params['ch4', 'color'] = '#0055ff'
view1.by_channel_params['ch5', 'color'] = '#ff5500'
view1.by_channel_params['ch6', 'color'] = 'red'

view1.by_channel_params['ch0', 'gain'] = 0.0005
view1.by_channel_params['ch1', 'gain'] = 0.002
view1.by_channel_params['ch2', 'gain'] = 0.002
view1.by_channel_params['ch3', 'gain'] = 0.001
view1.by_channel_params['ch4', 'gain'] = 0.002
view1.by_channel_params['ch5', 'gain'] = 0.002
view1.by_channel_params['ch6', 'gain'] = 0.001

view1.by_channel_params['ch0', 'offset'] = 4
view1.by_channel_params['ch1', 'offset'] = 1
view1.by_channel_params['ch2', 'offset'] = 1
view1.by_channel_params['ch3', 'offset'] = -3
view1.by_channel_params['ch4', 'offset'] = -6
view1.by_channel_params['ch5', 'offset'] = -6
view1.by_channel_params['ch6', 'offset'] = 8

#put this viewer in the main window
win.add_view(view1)

#Run
win.show()
app.exec()

print(len(CA1_i_onlystim[1][0]))
print(len(peaks_i_rec1), len(peaks_c_rec1))

debug True
QT_MODE PySide6
refresh duration for  0.0 s
save_all_settings
2571182
1310 1496


#### Recording 2

In [5]:
### REC 2 ###

### DETECT SWRs ON STIM ONLY DATA ###
#Do both hemispheres (can view both at same time) but only one recording at a time
for hemi in ["i", "c"]:
    if hemi == "i":
        threshold = 4   #number by which to multiply SD; 2 diff thresholds for 2 diff channels
    else:
        threshold = 4
    
    CA1_mean = globals()[f"CA1_{hemi}_means_sd"][2][0]
    CA1_sd = globals()[f"CA1_{hemi}_means_sd"][2][1]
    peaks, properties = find_peaks(globals()[f"CA1_{hemi}_onlystim"][2][0], prominence=1, width=20, height = CA1_mean+CA1_sd*threshold)

    properties["prominences"], properties["widths"]
    # SWR boundaries taken at 70% from peak of intensity. This means that the SWRs with small amplitude will be longer than the big ones.
    results_width = peak_widths(globals()[f"CA1_{hemi}_onlystim"][2][0], peaks, rel_height=0.7)
    peaks2 = peaks.reshape(len(peaks),1)
    npresults_width = np.array(results_width).reshape(4,-1)
    SWR_prop = np.append(peaks2, results_width).reshape(5,len(peaks2)).round()

    #Filter signal because is nice for SWR visualisation
    f_lowcut = 120.; f_hicut = 200.; fs = 1000; nyq = 0.5 * fs; N = 6
    Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction
    b, a = signal.butter(N, Wn, 'band')
    globals()[f"filt_CA1_{hemi}"] = signal.filtfilt(b, a, globals()[f"CA1_{hemi}"][2])
    globals()[f"peaks_{hemi}_rec2"] = peaks
    globals()[f"SWR_prop_{hemi}"] = SWR_prop



### LOOK AT THE RESULT AND JUDGE IT; REPEAT IF NECESSARY ###
app = mkQApp()

sample_rate = 1000.
t_start = 0.

combined = np.stack([CA1_i[2][bool_stimtimes_rec2], CA1_i_onlystim[2][0], filt_CA1_i[bool_stimtimes_rec2], CA1_c[2][bool_stimtimes_rec2], CA1_c_onlystim[2][0], filt_CA1_c[bool_stimtimes_rec2], EMGs[2][bool_stimtimes_rec2]], axis = 1)

#create 2 family scatters from theses 2 indexes
scatter_indexes = {0: peaks_i_rec2, 1: peaks_c_rec2}
#and assign them to some channels each
scatter_channels = {0: [0, 1], 1: [3, 4]}

source = AnalogSignalSourceWithScatter(combined, sample_rate, t_start, scatter_indexes, scatter_channels)

#Create the main window that can contain several viewers
win = MainViewer(debug=True, show_auto_scale=True)

#create a viewer for signal with TraceViewer
#connected to the signal source
view1 = TraceViewer(source=source)

#Parameters can be set in script
#view1.params['scale_mode'] = 'same_for_all'
view1.params['display_labels'] = True
#And also parameters for each channel
view1.by_channel_params['ch0', 'color'] = '#ffffff'
view1.by_channel_params['ch1', 'color'] = '#0055ff'
view1.by_channel_params['ch2', 'color'] = '#ff5500'
view1.by_channel_params['ch3', 'color'] = '#ffffff'
view1.by_channel_params['ch4', 'color'] = '#0055ff'
view1.by_channel_params['ch5', 'color'] = '#ff5500'
view1.by_channel_params['ch6', 'color'] = 'red'

view1.by_channel_params['ch0', 'gain'] = 0.0005
view1.by_channel_params['ch1', 'gain'] = 0.002
view1.by_channel_params['ch2', 'gain'] = 0.002
view1.by_channel_params['ch3', 'gain'] = 0.001
view1.by_channel_params['ch4', 'gain'] = 0.002
view1.by_channel_params['ch5', 'gain'] = 0.002
view1.by_channel_params['ch6', 'gain'] = 0.001

view1.by_channel_params['ch0', 'offset'] = 4
view1.by_channel_params['ch1', 'offset'] = 1
view1.by_channel_params['ch2', 'offset'] = 1
view1.by_channel_params['ch3', 'offset'] = -3
view1.by_channel_params['ch4', 'offset'] = -6
view1.by_channel_params['ch5', 'offset'] = -6
view1.by_channel_params['ch6', 'offset'] = 8

#put this viewer in the main window
win.add_view(view1)

#Run
win.show()
app.exec()

print(len(CA1_i_onlystim[2][0]))
print(len(peaks_i_rec2), len(peaks_c_rec2))

debug True
QT_MODE PySide6
refresh duration for  0.0 s
save_all_settings
3680040
1879 2179


#### Recording 3

In [6]:
### REC 3 ###

### DETECT SWRs ON STIM ONLY DATA ###
#Do both hemispheres (can view both at same time) but only one recording at a time
for hemi in ["i", "c"]:
    if hemi == "i":
        threshold = 4   #number by which to multiply SD; 2 diff thresholds for 2 diff channels
    else:
        threshold = 4
    
    CA1_mean = globals()[f"CA1_{hemi}_means_sd"][2][0]
    CA1_sd = globals()[f"CA1_{hemi}_means_sd"][2][1]
    peaks, properties = find_peaks(globals()[f"CA1_{hemi}_onlystim"][3][0], prominence=1, width=20, height = CA1_mean+CA1_sd*threshold)

    properties["prominences"], properties["widths"]
    # SWR boundaries taken at 70% from peak of intensity. This means that the SWRs with small amplitude will be longer than the big ones.
    results_width = peak_widths(globals()[f"CA1_{hemi}_onlystim"][3][0], peaks, rel_height=0.7)
    peaks2 = peaks.reshape(len(peaks),1)
    npresults_width = np.array(results_width).reshape(4,-1)
    SWR_prop = np.append(peaks2, results_width).reshape(5,len(peaks2)).round()

    #Filter signal because is nice for SWR visualisation
    f_lowcut = 120.; f_hicut = 200.; fs = 1000; nyq = 0.5 * fs; N = 6
    Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction
    b, a = signal.butter(N, Wn, 'band')
    globals()[f"filt_CA1_{hemi}"] = signal.filtfilt(b, a, globals()[f"CA1_{hemi}"][3])
    globals()[f"peaks_{hemi}_rec3"] = peaks
    globals()[f"SWR_prop_{hemi}"] = SWR_prop



### LOOK AT THE RESULT AND JUDGE IT; REPEAT IF NECESSARY ###
app = mkQApp()

sample_rate = 1000.
t_start = 0.

combined = np.stack([CA1_i[3][bool_stimtimes_rec3], CA1_i_onlystim[3][0], filt_CA1_i[bool_stimtimes_rec3], CA1_c[3][bool_stimtimes_rec3], CA1_c_onlystim[3][0], filt_CA1_c[bool_stimtimes_rec3], EMGs[3][bool_stimtimes_rec3]], axis = 1)

#create 2 family scatters from theses 2 indexes
scatter_indexes = {0: peaks_i_rec3, 1: peaks_c_rec3}
#and assign them to some channels each
scatter_channels = {0: [0, 1], 1: [3, 4]}

source = AnalogSignalSourceWithScatter(combined, sample_rate, t_start, scatter_indexes, scatter_channels)

#Create the main window that can contain several viewers
win = MainViewer(debug=True, show_auto_scale=True)

#create a viewer for signal with TraceViewer
#connected to the signal source
view1 = TraceViewer(source=source)

#Parameters can be set in script
#view1.params['scale_mode'] = 'same_for_all'
view1.params['display_labels'] = True
#And also parameters for each channel
view1.by_channel_params['ch0', 'color'] = '#ffffff'
view1.by_channel_params['ch1', 'color'] = '#0055ff'
view1.by_channel_params['ch2', 'color'] = '#ff5500'
view1.by_channel_params['ch3', 'color'] = '#ffffff'
view1.by_channel_params['ch4', 'color'] = '#0055ff'
view1.by_channel_params['ch5', 'color'] = '#ff5500'
view1.by_channel_params['ch6', 'color'] = 'red'

view1.by_channel_params['ch0', 'gain'] = 0.0005
view1.by_channel_params['ch1', 'gain'] = 0.002
view1.by_channel_params['ch2', 'gain'] = 0.002
view1.by_channel_params['ch3', 'gain'] = 0.001
view1.by_channel_params['ch4', 'gain'] = 0.002
view1.by_channel_params['ch5', 'gain'] = 0.002
view1.by_channel_params['ch6', 'gain'] = 0.001

view1.by_channel_params['ch0', 'offset'] = 4
view1.by_channel_params['ch1', 'offset'] = 1
view1.by_channel_params['ch2', 'offset'] = 1
view1.by_channel_params['ch3', 'offset'] = -3
view1.by_channel_params['ch4', 'offset'] = -6
view1.by_channel_params['ch5', 'offset'] = -6
view1.by_channel_params['ch6', 'offset'] = 8

#put this viewer in the main window
win.add_view(view1)

#Run
win.show()
app.exec()

print(len(CA1_i_onlystim[3][0]))
print(len(peaks_i_rec3), len(peaks_c_rec3))

debug True
QT_MODE PySide6
refresh duration for  0.0 s
save_all_settings
1871881
944 1820


#### Remove bad events

In [ ]:
#If go through recording with visualiser and see obviously wrong event, write the approximate timing of them here (can be done in parallel with open ephyviewer)
# Once everything to exclude is written down for all recordings, run next cell (ONLY ONCE or run all thresholding-ephyviewer cells again)

events_toremove_CA1_i_rec0 = []
events_toremove_CA1_c_rec0 = []

events_toremove_CA1_i_rec1 = []
events_toremove_CA1_c_rec1 = []

events_toremove_CA1_i_rec2 = []
events_toremove_CA1_c_rec2 = []

#etc.

In [ ]:
#Run this cell to exclude flagged events
# Will go through the list of flagged events and remove the event in the original list (peaks) that is closest in time to the noted timestamp

def closest(list, Number):
    aux = []
    for valor in list:
        aux.append(abs(Number*1000-valor))

    return list[aux.index(min(aux))]

for hemi in ["i", "c"]:
    for rec_nb in range(len(recordings)):
        events_toremove = globals()[f"events_toremove_CA1_{hemi}_rec{rec_nb}"]
        peaks = globals()[f"peaks_{hemi}_rec{rec_nb}"]

        print( hemi, rec_nb)
        print(len(peaks))
        print(len(peaks)-len(events_toremove))

        for remove in events_toremove:
            event = closest(peaks, remove)
            mask = peaks != event
            peaks = peaks[mask]

        print(len(peaks))   #to check that right number of events was excluded

### Count events in right bins

In [7]:
# THis will take all events from the peaks variables and place them in the correct bins: correct recording, hemisphere CA1, and stim freq

#When working with CA1_onlystim (shortened version of recording), times of events don't fit anymore, so to get back real times: value in peaks tuple = index of onlystim array; and at that index (last row) can be found time = index of realtime CA1

#Meaning of dimensions of event_count variables: (CA1 ipsi & contra; stim freqs 2-90 Hz; bins 5 before, 5 during, 5 after stim, & 10s after stim with 0.5s per bin)
event_count_mid = np.zeros((2, 13, 35))
event_count_vb = np.zeros((2, 13, 35))

all_stim_freq = (2,4,6,8,10,12,14,16,18,20,40,60,90)
total_count = 0


for stim_region in stim_regions:
    for rec_nb in range(len(recordings)):
        stim_times = globals()[f"stim_times_{stim_region}_rec{rec_nb}"]
        for f, stim_freq in enumerate(all_stim_freq):
            for stim_nb in range(0, len(stim_times), 10):   #every 10th stim because blocks of 10. To be optimised.
                stim_start = stim_times[stim_nb, 0]
                stim_duration = stim_times[stim_nb+9, 1] - stim_start
                bins = np.concatenate((np.arange(stim_start-stim_duration, stim_start+2*stim_duration, stim_duration/5), np.arange(stim_start+2*stim_duration, stim_start+2*stim_duration+10000, 500)))
                for h, hemi in enumerate(["i", "c"]): 
                    times_wrong = globals()[f"peaks_{hemi}_rec{rec_nb}"]
                    CA1_onlystim = globals()[f"CA1_{hemi}_onlystim"][rec_nb]
                    times = []
                    for t in times_wrong:
                        times.append(CA1_onlystim[1][t])

                    #Function created by ChatGPT
                    def find_bin_indices(bins, times):
                        times = np.array(times)
                        indices = np.searchsorted(bins, times, side='right') - 1
                        valid_indices = (indices >= 0) & (indices < len(bins) - 1)
                        indices = indices[valid_indices]
                        times = times[valid_indices]
                        in_bin = (times >= bins[indices]) & (times < bins[indices + 1])
                        result_indices = indices[in_bin]
                        return result_indices

                    bin_indices = find_bin_indices(bins, times)

                    #Append count only if stim we're looking at fits with stim freq in duration (with less precision necessary if high freq)
                    if stim_freq<30:
                        if round(9000/(stim_duration-20)) == stim_freq:
                            for b in bin_indices:
                                globals()[f"event_count_{stim_region}"][h, f, b] += 1
                            total_count += 1
                    else:
                        if round(9000/(stim_duration-20), -1) == stim_freq:
                            for b in bin_indices:
                                globals()[f"event_count_{stim_region}"][h, f, b] += 1
                            total_count +=1
                    #print(globals()[f"event_count_{stim_region}"][rec_nb])

Comments:
ipsi very good, but contra excellent. will pick contra

In [8]:
event_count_both = np.concatenate((event_count_mid, event_count_vb))
np.save(f"SWR count mouse{mouse} mid-vb.npy", event_count_both)

#format of numpy array:
#Meaning of dimensions: (CA1 ipsi & contra; stim freqs 2-90 Hz; bins 5 before, 5 during, 5 after stim, & 10s after stim with 0.5s per bin) (see above)
# and mid - CA1 ipsi contra; vb - CA1 ipsi contra


In [16]:
#Quick-check result

before = np.sum(event_count_mid[0, 5:9, :5])
during = np.sum(event_count_mid[0, 5:9, 5:10])
after = np.sum(event_count_mid[0, 5:9, 10:15])
after2 = np.sum(event_count_mid[0, 5:9, 16:17])
after3 = np.sum(event_count_mid[0, 5:9, 17:18])
after4 = np.sum(event_count_mid[0, 5:9, 18:19])
after5 = np.sum(event_count_mid[0, 5:9, 19:20])
after6 = np.sum(event_count_mid[0, 5:9, 20:21])
after7 = np.sum(event_count_mid[0, 5:9, 21:22])
after8 = np.sum(event_count_mid[0, 5:9, 22:23])

print(before, during, after, after2, after3, after4, after5, after6, after7, after8)

45.0 27.0 37.0 27.0 36.0 37.0 32.0 32.0 31.0 42.0
